# NetSurfP 3.0

The purpose of this notebook is to improve NetSurfP 2.0 to use pretrained model embeddings

**Load libraries**

The different libraries are initialized and pytorch is either configured to use the CPU or an available GPU.

In [1]:
import pylab
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

import optuna

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, PackedSequence
from torch.autograd import Variable

# custom packages in parent folder
import os
import sys

os.environ["UTILS"] = "/home/projects/ht3_aim/people/erikie/NSPThesis/notebooks/utils"

computerome = os.getenv("UTILS", "..")
sys.path.append(computerome)

from early_stopping import EarlyStopping
from masked_batch_norm import MaskedBatchNorm1d

# debugging
import pdb

Setup CUDA device

In [2]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

Using device: cuda
Tesla V100-PCIE-16GB


## 1. Preparation of the data

Each dataset is compressed as a numpy zip and the dimensions of the dataset are as following: 

**[sequence, position, label]**

The labels are as following in the data (shown by index):
- [0:20] Amino Acids (sparse encoding) (Unknown residues are stored as an all-zero vector)
- [20:50] hmm profile
- [50] Seq mask (1 = seq, 0 = empty)
- [51] Disordered mask (0 = disordered, 1 = ordered)
- [52] Evaluation mask (For CB513 dataset, 1 = eval, 0 = ignore)
- [53] ASA (isolated)
- [54] ASA (complexed)
- [55] RSA (isolated)
- [56] RSA (complexed)
- [57:65] Q8 GHIBESTC (Q8 -> Q3: HHHEECCC)
- [65:67] Phi+Psi
- [67] ASA_max

### Loading the datasets

In [3]:
data_dir = "../../data/nsp3/training_data/"

train_hhblits = np.load(data_dir + "small_esm1b_train_hhblits.npz")
#CB513_hhblits = np.load(data_dir + "CB513_HHblits.npz")
#TS115_hhblits = np.load(data_dir + "TS115_HHblits.npz")

CASP12_hhblits = np.load(data_dir + "esm1b_CASP12_HHblits.npz")

#train_mmseqs = np.load("../../data/nsp2/training_data/Train_MMseqs.npz")
#CB513_mmseqs = np.load("../../data/nsp2/training_data/CB513_MMseqs.npz")
#TS115_mmseqs = np.load("../../data/nsp2/training_data/TS115_MMseqs.npz")
#CASP12_mmseqs = np.load("../../data/nsp2/training_data/CASP12_MMseqs.npz")

### Custom Data loader class

A data loader class is created to load the NSP data into a DataLoader object. The DataLoader is a pytorch object, thats used to feed the data as batches. Which will be usefull when training and evaluating the NSP model. The NSPData class divides the dataset into input data and target labels.

In [4]:
class NSPData(Dataset):
    def __init__(self, dataset, indices = False):
        """ Constructor
        Args:
            X (np.array): The array that contains the training data
            y (np.array): The array that contains the test data
        """

        self.data = torch.tensor(dataset[:, :, :20]).float()
        self.embeddings = torch.tensor(dataset[:, :, 68:]).float()
        self.targets = torch.tensor(dataset[:, :, 50:68]).float()
        self.lengths = torch.tensor([sum(target[:, 0] == 1) for target in self.targets])
        
        # add the one hot encoding to the embeddings
        self.embeddings = torch.cat([self.data[:, :, :20], self.embeddings], dim=2)
        
        self.unknown_nucleotide_mask()
        
    def unknown_nucleotide_mask(self):
        """ Augments the target with a unknown nucleotide mask
            by finding entries that don't have any residue
        """
        
        # creates a mask based on the one hot encoding
        unknown_nucleotides = torch.max(self.data[:, :, :20], dim=2)
        unknown_nucleotides = unknown_nucleotides[0].unsqueeze(2)
        
        # merge the mask to first position of the targets
        self.targets = torch.cat([self.targets, unknown_nucleotides], dim=2)
        
    def __getitem__(self, index):
        """ Returns train and test data at an index
        Args:
            index (int): Index at the array
        """
        X = self.embeddings[index]
        y = self.targets[index]
        lengths = self.lengths[index]
        
        return X, y, lengths
    
    def __len__(self):
        """Returns the length of the data"""
        return len(self.data)

# Holdout

It is important to have a training and validation set when training to evaluate the training performance. Therefore, a function that splits the dataset into training and evaluation is created for that purpose.

In [5]:
def split_dataset(batch_size, dataset):
    """ Splits the dataset into train and validation
    Args:
        batch_size (int): size of each batch
        dataset (np.array): dataset containing training data
        validation_fraction (float): the size of the validation set as a fraction
    """
    
    num_train = len(dataset)
    train_indices = np.array(range(num_train))
    validation_indices = np.random.choice(train_indices, int(num_train*0.05), replace=False)
    
    train_indices = np.delete(train_indices, validation_indices)
    
    # subset the dataset
    train_idx, valid_idx = train_indices, validation_indices
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    
    training_set = DataLoader(NSPData(dataset), sampler=train_sampler, batch_size=batch_size)
    validation_set = DataLoader(NSPData(dataset), sampler=valid_sampler, batch_size=batch_size)
    
    return training_set, validation_set

**Instanciate data into the data loader class**

The training datasets are splitted into training and validation for training. Whereas the test sets are simply loaded, since they are independent datasets. The batch sizes is set to 15. The default training/validation split is 80/20.

In [6]:
batch_size = 15

In [7]:
# training and validation sets
#train_hhblits = DataLoader(NSPData(train_hhblits), batch_size=batch_size, shuffle=True) 
train_hhblits = split_dataset(batch_size, train_hhblits['data'])
#train_mmseqs = DataLoader(NSPData(train_mmseqs), batch_size=batch_size) #split_dataset(batch_size, train_mmseqs)

# test sets
#CB513_hhblits = DataLoader(NSPData(CB513_hhblits), batch_size=len(CB513_hhblits['data']))
#TS115_hhblits = DataLoader(NSPData(TS115_hhblits), batch_size=len(TS115_hhblits['data']))
CASP12_hhblits = DataLoader(NSPData(CASP12_hhblits['data']), batch_size=len(CASP12_hhblits['data']))

#CB513_mmseqs = DataLoader(NSPData(CB513_mmseqs), batch_size=batch_size)
#TS115_mmseqs = DataLoader(NSPData(TS115_mmseqs), batch_size=batch_size)
#CASP12_mmseqs = DataLoader(NSPData(CASP12_mmseqs), batch_size=batch_size)

## 2. The NSP Classifier

The input layer consists of one-hot encoded sequences (20 features amino acids) and a full HMM profile (20 features for amino acid profile, 7 features of state transition probabilities and 3 features for local alignment diversity) The input layer is followed by two layers of 1D CNN layers, that consist of 32 filters with a size of 129 and 257. Whereas the output of the last 1D CNN is concatenated with the initial input features. These residuals connections are used to achieve a better backpropagation without gradient degradation and a deeper network. The concatenated input+residuals is then applied to 2 bidirectional long  short-term memory (LSTM) layers with 1024 nodes, that outputs 2048 hidden neurons. The hidden neurons output is input to a fully connected (FC) layer to predict the 18 classes (RSA, SS8, SS3, φ, ψ, and disorder).

In [8]:
class NSP_Network(nn.Module):
    def __init__(self, init_channels, n_hidden, cnn_output, params):
        """ Initializes the model with the required layers
        Args:
            init_channels: The size of the incoming feature vector
            n_hidden: The amount of hidden neurons in the bidirectional lstm
        """
        super(NSP_Network, self).__init__()

        # CNN block
        self.conv1 = nn.Sequential(*[
            nn.Dropout(p=0.5),
            nn.Conv1d(in_channels=init_channels, out_channels=cnn_output, kernel_size=129, padding=64),
            nn.ReLU(),
        ])

        self.conv2 = nn.Sequential(*[
            nn.Dropout(p=0.5),
            nn.Conv1d(in_channels=init_channels, out_channels=cnn_output, kernel_size=257, padding=128),
            nn.ReLU(),
        ])
        
        self.batch_norm = MaskedBatchNorm1d(cnn_output*2+init_channels)
        
        decrease_value = 512
        self.decrease = nn.Linear(in_features=cnn_output*2+init_channels, out_features=decrease_value)

        # LSTM block
        #self.lstm = nn.LSTM(input_size=init_channels+cnn_output*2, hidden_size=n_hidden, batch_first=True, \
        #                    num_layers=2, bidirectional=True, dropout=0.5)
        
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=decrease_value, nhead=8, dropout=0.5)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=params['num_layers_encoder'])
        
        # add dropout to last layer
        self.lstm_dropout = nn.Dropout(p=0.5)

        # output block
        self.ss8 = nn.Sequential(*[
            nn.Linear(in_features=decrease_value, out_features=8),
            #nn.Softmax(),
        ])
        self.ss3 = nn.Sequential(*[
            nn.Linear(in_features=decrease_value, out_features=3),
            #nn.Softmax(),
        ])     
        self.disorder = nn.Sequential(*[
            nn.Linear(in_features=decrease_value, out_features=2),
            #nn.Softmax(),
        ])
        self.rsa = nn.Sequential(*[
            nn.Linear(in_features=decrease_value, out_features=1),
            nn.Sigmoid()
        ])
        self.phi = nn.Sequential(*[
            nn.Linear(in_features=decrease_value, out_features=2),
            nn.Tanh()
        ])
        self.psi = nn.Sequential(*[
            nn.Linear(in_features=decrease_value, out_features=2),
            nn.Tanh()
        ])
        
    def forward(self, x, t, mask):
        """ Forwards the input through each layer in the model
        Args:
            x: input data containing sequences
            lengths: list containing each sequence length
            mask: data containing the sequence padding mask
        """
    
        _, length, _ = x.size()
        
        # calculate the residuals
        x = x.permute(0,2,1)
        
        r1 = self.conv1(x)
        r2 = self.conv2(x)
                
        # concatenate channels from residuals and input
        x = torch.cat([x, r1, r2], dim=1)
                                 
        # normalize
        x = self.batch_norm(x, mask.unsqueeze(1))
        
        x = x.permute(0,2,1)
        x = self.decrease(x)
        
        # calculate double layer bidirectional lstm
        
        #x = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        #x, _ = self.lstm(x)
        #x, _ = pad_packed_sequence(x, total_length=length, batch_first=True)
        
        x = self.transformer_encoder(x)
        x = torch.nan_to_num(x)
        
        x = self.lstm_dropout(x)
        
        # hidden neurons to classes
        ss8 = self.ss8(x)
        ss3 = self.ss3(x)
        disorder = self.disorder(x)
        rsa = self.rsa(x)
        phi = self.phi(x)
        psi = self.psi(x)

        return [ss8, ss3, disorder, rsa, phi, psi]

## 3. Loss function

To train the model, loss functions are required. The tasks SS8, SS3 and disorder uses cross entropy loss function and RSA, φ and ψ use mean squared error loss.

Weights were adjusted so each loss contribution was approximately equal by using homoscedastic uncertainty and optimizing the log variances for each prediction. (https://arxiv.org/abs/1705.07115)

In the forwarding of the MultiTaskLoss, filters are used to remove extrapolated zeros or disordered regions from the calculated losses.

The indexes for the labels are described in the data preparation section.

In [9]:
class MultiTaskLoss(nn.Module):
    """ Weighs multiple loss functions by considering the 
        homoscedastic uncertainty of each task """
    
    def __init__(self):
        super(MultiTaskLoss, self).__init__()
        
    def mse(self, outputs, labels, mask):
        loss = torch.square(outputs - labels) * mask
        return torch.sum(loss) / torch.sum(mask)
    
    def cross_entropy(self, outputs, labels, mask):
        labels = labels.clone()
        labels[mask == 0] = -999
        
        return nn.CrossEntropyLoss(ignore_index=-999)(outputs, labels.long())
        
    def ss8(self, outputs, labels, mask):
        labels = torch.argmax(labels[:, :, 7:15], dim=2)
        outputs = outputs[0].permute(0, 2, 1)
        
        return self.cross_entropy(outputs, labels, mask)
        
    def ss3(self, outputs, labels, mask):
        structure_mask = torch.tensor([0,0,0,1,1,2,2,2]).to(device)

        labels = torch.max(labels[:, :, 7:15] * structure_mask, dim=2)[0].long()
        outputs = outputs[1].permute(0, 2, 1)
        
        return self.cross_entropy(outputs, labels, mask)
        
    def disorder(self, outputs, labels, mask):
        # apply the disorder loss
        labels = labels[:, :, 1].unsqueeze(2)
        labels = torch.argmax(torch.cat([labels, 1-labels], dim=2), dim=2)
        
        outputs = outputs[2].permute(0, 2, 1)
        
        return self.cross_entropy(outputs, labels, mask)
        
    def rsa(self, outputs, labels, mask):
        labels = labels[:, :, 5].unsqueeze(2)
        outputs = outputs[3]
        
        mask = mask.unsqueeze(2)
        
        return self.mse(outputs, labels, mask)
        
    def phi(self, outputs, labels, mask):
        labels = labels[:, :, 15].unsqueeze(2)
        outputs = outputs[4]
        
        mask = mask * (labels != 360).squeeze(2).int()
        mask = torch.cat(2*[mask.unsqueeze(2)], dim=2)
        
        loss = self.mse(outputs.squeeze(2), torch.cat((torch.sin(dihedral_to_radians(labels)), torch.cos(dihedral_to_radians(labels))), dim=2).squeeze(2), mask)
        return loss
        
    def psi(self, outputs, labels, mask):
        labels = labels[:, :, 16].unsqueeze(2)
        outputs = outputs[5]
        
        mask = mask * (labels != 360).squeeze(2).int()
        mask = torch.cat(2*[mask.unsqueeze(2)], dim=2)
        
        loss = self.mse(outputs.squeeze(2), torch.cat((torch.sin(dihedral_to_radians(labels)), torch.cos(dihedral_to_radians(labels))), dim=2).squeeze(2), mask)
        return loss

    def forward(self, outputs, labels, weighted=True):
        """ Forwarding of the multitaskloss input
        Args:
            outputs (torch.tensor): output data from model
            labels (torch.tensor): corresponding labels for the output
        """

        # filters
        zero_mask = labels[:, :, 0]
        disorder_mask = labels[:, :, 1]
        unknown_mask = labels[:, :, -1]
        
        # weighted losses
        ss8 = self.ss8(outputs, labels, zero_mask) * 1
        ss3 = self.ss3(outputs, labels, zero_mask) * 5
        dis = self.disorder(outputs, labels, zero_mask) * 5
        rsa = self.rsa(outputs, labels, zero_mask * disorder_mask * unknown_mask) * 100
        phi = self.phi(outputs, labels, zero_mask * disorder_mask * unknown_mask) * 5
        psi = self.psi(outputs, labels, zero_mask * disorder_mask * unknown_mask) * 5
        
        loss = torch.stack([ss8, ss3, dis, rsa, phi, psi])
        
        return loss.sum()

### Utils

The model trains phi and psi into each their vector that will contain calculated cos and sin values from the angle. Pytorch works with radians, so these helper functions helps converting dihedral angle values into radians and cos and sin radian values back to a dihedral angle.

In [10]:
def dihedral_to_radians(angle):
    """ Converts angles to radians
    Args:
        angles (1D Tensor): vector with angle values
    """
    return angle*np.pi/180
    
def arctan_dihedral(sin, cos):
    """ Converts sin and cos back to diheral angles
    Args:
        sin (1D Tensor): vector with sin values 
        cos (1D Tensor): vector with cos values
    """
    result = torch.where(cos >= 0, torch.arctan(sin/cos), torch.arctan(sin/cos)+np.pi)
    result = torch.where((sin <= 0) & (cos <= 0), result-np.pi*2, result)
    
    return result*180/np.pi

## 5. Training

Instanciating the model, criterion and optimizer

In [11]:
def init_model(initial_channels, hidden_neurons, cnn_output, learning_rate, params):
    """ Initializes a model, criterion and optimizer 
    Args:
        initial_channels (int): amount of initial inputs for the model
        hidden_neurons (int): amount of hidden neurons in the model
    """
    nsp_net = NSP_Network(initial_channels, hidden_neurons, cnn_output, params)
    criterion = MultiTaskLoss()

    # enable cuda on model and criterion if possible
    if device.type != "cpu":
        nsp_net.cuda(device)
        criterion.cuda(device)

    # optimizer for model and criterion
    optimizer = optim.Adam([{"params" : criterion.parameters()}, {"params" : nsp_net.parameters()}], lr=learning_rate)
    
    return nsp_net, criterion, optimizer

The model is trained by iterating through the training dataset for an amount of epochs. The model is trained using backpropagation using the multiple task loss.

In [12]:
def training(epochs, model, criterion, optimizer, dataset):
    # iterate over the dataset multiple times
    training_loss = []
    validation_loss = []
        
    train, test = dataset

    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-2, steps_per_epoch=len(train), epochs=epochs)

    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=3, verbose=True)

    for epoch in range(epochs):
        #print('Epoch:', epoch + 1,' of ', epochs)

        # training of the model 
        running_loss = 0.0
        for i, data in enumerate(train, 0):

            # move data tensors to GPU if possible
            inputs, labels, lengths = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            padding = labels[:, :, 0]
            outputs = model(inputs, labels, padding)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            # backpropagation by custom criterion
            loss = criterion(outputs, labels)
            loss.backward()

            optimizer.step()
            #scheduler.step()

            running_loss += loss.item()

        training_loss.append(running_loss / len(train))
        #print("Training loss: ", round(training_loss[epoch], 3))

        #validation of the model
        running_loss = 0.0
        with torch.no_grad():
            for i, data in enumerate(test, 0):
                
                # move data tensors to GPU if possible
                inputs, labels, lengths = data
                inputs, labels = inputs.to(device), labels.to(device)
                
                padding = labels[:, :, 0]
                outputs = model(inputs, labels, padding)

                loss = criterion(outputs, labels)

                running_loss += loss.item()

        validation_loss.append(running_loss / len(test))
        #print("Validation loss: ", round(validation_loss[epoch], 3))

        early_stopping(validation_loss[epoch], model)

        if early_stopping.early_stop:
            #print("Early stopping")
            break

    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))
        
    #print('Finished Training')
    
    return training_loss, validation_loss

The training loss and validation loss are visualized to evaluate the training performance.

In [13]:
def plot_loss(training_loss, validation_loss, title):
    pylab.plot(training_loss)
    pylab.plot(validation_loss)
    pylab.xlabel('Epochs')
    pylab.ylabel('Loss [sum]')
    pylab.legend(('Training loss', 'Validation loss'))
    pylab.title(title)
    pylab.show()

Start the training

In [ ]:
epochs = 100

initial_channels = 1300
cnn_out = 32
hidden_neurons = 57
learning_rate = 1e-3

def objective(trial):
    params = {
        #"dropout_rate" : trial.suggest_uniform("dropout_rate", 0.4, 0.5),
        "num_layers_encoder": trial.suggest_int("num_layers_encoder", 1, 6),
        "learning_rate": trial.suggest_loguniform('learning_rate', 1e-6, 1e-1),
    }
    
    hyperoptim = init_model(initial_channels, hidden_neurons, cnn_out, params['learning_rate'], params)
    
    training_loss, evaluation_loss = training(epochs, hyperoptim[0], hyperoptim[1], hyperoptim[2], train_hhblits)
    
    return min(evaluation_loss)
    
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Best trial")
best_parameters = study.best_trial

print(best_parameters.values)
print(best_parameters.params)

[I 2021-03-15 13:11:13,457] A new study created in memory with name: no-name-91f35318-b5e2-4ad0-8709-35bcfaaf0fa3


Validation loss decreased (inf --> 24.998795).  Saving model ...
Validation loss decreased (24.998795 --> 21.521193).  Saving model ...
Validation loss decreased (21.521193 --> 19.974209).  Saving model ...
Validation loss decreased (19.974209 --> 18.782351).  Saving model ...


In [ ]:
epochs = 100

initial_channels = 1300
cnn_out = 32
hidden_neurons = 57
learning_rate = 1e-3

#Trial 19 finished with value: 17.18759536743164 and parameters: {'dropout_rate': 0.10882566406017377, 'num_layers_encoder': 2, 'learning_rate': 0.0003437625388266798}. Best is trial 18 with value: 16.15894889831543.

nsp_hhblits = init_model(initial_channels, hidden_neurons, cnn_out, learning_rate)

training_loss, evaluation_loss = training(epochs, nsp_hhblits[0],
                                                    nsp_hhblits[1],
                                                    nsp_hhblits[2],
                                                    train_hhblits)

plot_loss(training_loss, evaluation_loss, "NSP HHblits training and validation loss")

#training_loss, evaluation_loss = training(epochs, nsp_mmseqs[0], 
#                                          nsp_mmseqs[1], 
#                                          nsp_mmseqs[2],
#                                          train_mmseqs)

#plot_loss(training_loss, evaluation_loss, "NSP MMseqs training and validation loss")

## 5. Evaluation

Evaluation functions are created to evaluate the models performance.

In [ ]:
def fpr(pred, labels):
    """ False positive rate
    Args:
        inputs (1D Tensor): vector with predicted binary numeric values
        labels (1D Tensor): vector with correct binary numeric values
    """
    fp = sum((pred == 1) & (labels == 0))
    tn = sum((pred == 0) & (labels == 0))
    
    return (fp/(fp+tn)).item()

def fnr(pred, labels):
    """ False negative rate
    Args:
        inputs (1D Tensor): vector with predicted binary numeric values
        labels (1D Tensor): vector with correct binary numeric values
    """
    fn = sum((pred == 0) & (labels == 1))
    tp = sum((pred == 1) & (labels == 1))
    
    return (fn/(fn+tp)).item()

def mcc(pred, labels):
    """ Mathews correlation coefficient
    Args:
        inputs (1D Tensor): vector with predicted binary numeric values
        labels (1D Tensor): vector with correct binary numeric values
    """
    fp = sum((pred == 1) & (labels == 0))
    tp = sum((pred == 1) & (labels == 1))
    fn = sum((pred == 0) & (labels == 1))
    tn = sum((pred == 0) & (labels == 0))
    
    return ((tp*tn-fp*fn)/torch.sqrt(((tp+fp)*(fn+tn)*(tp+fn)*(fp+tn)).float())).item()

def pcc(pred, labels):
    """ Pearson correlation coefficient
    Args:
        inputs (1D Tensor): vector with predicted numeric values
        labels (1D Tensor): vector with correct numeric values
    """
    x = pred - torch.mean(pred)
    y = labels - torch.mean(labels)
    
    return (torch.sum(x * y) / (torch.sqrt(torch.sum(x ** 2)) * torch.sqrt(torch.sum(y ** 2)))).item()

def mae(pred, labels):
    """ Mean absolute error
    Args:
        inputs (1D Tensor): vector with predicted numeric values
        labels (1D Tensor): vector with correct numeric values
    """
    err = torch.abs(labels - pred)
    return torch.mean(torch.fmin(err, 360-err)).item()

def accuracy(pred, labels):
    """ Accuracy coefficient
    Args:
        inputs (1D Tensor): vector with predicted integer values
        labels (1D Tensor): vector with correct integer values
    """
    
    return (sum((pred == labels)) / len(labels)).item()

Seperate datasets are used to evaluate the model. Here is each class evaluated and the prediction benchmarks for the trained model are found.

In [ ]:
def evaluate_ss8(outputs, labels, mask):
    labels = torch.argmax(labels[:, :, 7:15], dim=2)[mask == 1]
    outputs = torch.argmax(outputs, dim=2)[mask == 1]
        
    return accuracy(outputs, labels)
    
def evaluate_ss3(outputs, labels, mask):
    structure_mask = torch.tensor([0,0,0,1,1,2,2,2])

    labels = torch.max(labels[:, :, 7:15] * structure_mask, dim=2)[0].long()[mask == 1]
    outputs = torch.argmax(outputs, dim=2)[mask == 1]
        
    return accuracy(outputs, labels)
    
def evaluate_dis(outputs, labels, mask, metric="fpr"):
    labels = labels[:, :, 1].unsqueeze(2)
    labels = torch.argmax(torch.cat([labels, 1-labels], dim=2), dim=2)[mask == 1]
    outputs = torch.argmax(outputs, dim=2)[mask == 1]

    if metric == "fpr":
        return fpr(outputs, labels)
    else:
        return mcc(outputs, labels)
    
def evaluate_rsa(outputs, labels, mask):
    labels = labels[:, :, 5].unsqueeze(2)[mask == 1]
    outputs = outputs[mask == 1]
        
    return pcc(outputs, labels)

def evaluate_asa(outputs, labels, mask):
    outputs = (outputs * labels[:, :, 17].unsqueeze(2))[mask == 1]
    labels = labels[:, :, 3].unsqueeze(2)[mask == 1]
    
    return pcc(outputs, labels)

def evaluate_phi(outputs, labels, mask):    
    labels = labels[:, :, 15]
    
    mask = mask * (labels != 360).int()
    labels = labels[mask == 1]
    
    outputs = arctan_dihedral(outputs[:, :, 0], outputs[:, :, 1])[mask == 1]
    
    return mae(outputs, labels)
    
def evaluate_psi(outputs, labels, mask):
    labels = labels[:, :, 16]
    
    mask = mask * (labels != 360).int()
    labels = labels[mask == 1]
    
    outputs = arctan_dihedral(outputs[:, :, 0], outputs[:, :, 1])[mask == 1]
    
    return mae(outputs, labels)

def evaluation(model, dataset):    
    # iterate through the evaluation dataset
    with torch.no_grad():
        for data in dataset:
            # move data tensors to GPU if possible
            inputs, labels, lengths = data
            
            # add evaluation mask to the masks
            evaluation_mask = labels[:, :, 2]
            
            zero_mask = labels[:, :, 0] * evaluation_mask
            disorder_mask = labels[:, :, 1]
            unknown_mask = labels[:, :, -1]
            
            # get predictions
            model = model.to('cpu')
            predictions = model(inputs, lengths, zero_mask) # predict values
            
            # move predictions to cpu
            for i in range(len(predictions)):
                predictions[i] = predictions[i].cpu()
                
            labels = labels.cpu()
            zero_mask = zero_mask.cpu()
            disorder_mask = disorder_mask.cpu()
            unknown_mask = unknown_mask.cpu()
            
            # evaluate
            ss8 = evaluate_ss8(predictions[0], labels, zero_mask)
            ss3 = evaluate_ss3(predictions[1], labels, zero_mask)
            dis_fpr = evaluate_dis(predictions[2], labels, zero_mask, metric="fpr")
            dis_mcc = evaluate_dis(predictions[2], labels, zero_mask, metric="mcc")
            rsa = evaluate_rsa(predictions[3], labels, zero_mask * disorder_mask * unknown_mask)
            asa = evaluate_asa(predictions[3], labels, zero_mask * disorder_mask * unknown_mask)
            phi = evaluate_phi(predictions[4], labels, zero_mask * disorder_mask * unknown_mask)
            psi = evaluate_psi(predictions[5], labels, zero_mask * disorder_mask * unknown_mask)
            
            print("RSA (PCC): {}".format(round(rsa, 3)))
            print("ASA (PCC): {}".format(round(asa, 3)))
            print("SS8 [Q8]: {}".format(round(ss8, 3)))
            print("SS3 [Q3]: {}".format(round(ss3, 3)))
            print("Disorder (MCC): {}".format(round(dis_mcc, 2)))
            print("Disorder (FNR): {}".format(round(dis_fpr, 3)))
            print("Phi (MAE): {}".format(round(phi, 3)))
            print("Psi (MAE): {}".format(round(psi, 3)))

The models are evaluated

In [ ]:
## print("Evaluation HHblits...")

#print("\nCB513")
#evaluation(nsp_hhblits[0], CB513_hhblits)
#print("\nTS115")
#evaluation(nsp_hhblits[0], TS115_hhblits)
print("\nCASP12")
evaluation(nsp_hhblits[0], CASP12_hhblits)

#print("\nEvaluation MMseqs...")

#print("\nCB513")
#evaluation(nsp_mmseqs[0], CB513_hhblits)
#print("\nTS115")
#evaluation(nsp_mmseqs[0], TS115_hhblits)
#print("\nCASP12")
#evaluation(nsp_mmseqs[0], CASP12_hhblits)